# Widgets

This example uses slider widgets to adjust parameters

In [1]:
import dpsim
import math, cmath
import time
import os

import matplotlib as plt
from dataprocessing.dpsim import *
from dataprocessing.plottools import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
def simulate(dt, dur, pha, mag, r1, r2, l, c):    
    gnd = dpsim.dp.Node.GND()
    n1 =  dpsim.dp.Node("n1")
    n2 =  dpsim.dp.Node("n2")

    cs = dpsim.dp.ph1.CurrentSource("cs", [gnd, n1], I_ref=cmath.rect(mag, pha))
    r1 = dpsim.dp.ph1.Resistor("r_1", [n1, gnd], R=r1)
    c1 = dpsim.dp.ph1.Capacitor("c_1", [n1, n2], C=c)
    l1 = dpsim.dp.ph1.Inductor("l_1", [n2, gnd], L=l*1e-3)
    r2 = dpsim.dp.ph1.Resistor("r_2", [n2, gnd], R=r2*1e-3)

    sys = dpsim.SystemTopology(50, [gnd, n1, n2], [cs, r1, c1, l1, r2])
    sim = dpsim.Simulation("Widgets", sys, duration=dur, timestep=dt)
    
    logger = dpsim.Logger("Widgets")
    logger.log_attribute(n1, "v") # v1
    logger.log_attribute(n2, "v") # v2
    logger.log_attribute(cs, "i_intf") # i12
    logger.log_attribute(c1, "i_intf") # i34
    sim.add_logger(logger)
    
    sim.start()
    while sim.state != 9:
        time.sleep(0.01)
        
    os.system('head -n-1 Logs/Widgets.csv > Logs/Widgets_fixed.csv')
        
    results = read_timeseries_dpsim('Logs/Widgets_fixed.csv')
    for l in [ 'n1.v', 'n2.v', 'cs.i_intf', 'c_1.i_intf' ]:
        emt = results[l].dynphasor_shift_to_emt(l + '_emt', 50)
        plot_timeseries(1, emt)

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

output = interactive(simulate,
    dt  = widgets.FloatText(description="Timestep [s]", value=1e-3, min=1e-3, max=1),
    dur = widgets.FloatText(description="Duration [s]", value=0.1, min=0, max=10),
    pha = widgets.FloatSlider(description="Phase [rad]", min=-math.pi, max=math.pi),
    mag = widgets.FloatSlider(description="Magnitude [V]", value=10, min=0, max=100),
    r1  = widgets.FloatSlider(description="Resistance [Ohm]", value=1, min=0.1, max=10),
    r2  = widgets.FloatSlider(description="Resistance [Ohm]", value=1, min=0.1, max=10),
    l   = widgets.FloatSlider(description="Inductance [H]", value=1, min=1, max=10),
    c   = widgets.FloatSlider(description="Capactance [F]", value=1, min=1, max=10),
    continuous_update=False
)

last = output.children[-1]
last.layout.height = '400px'
output

interactive(children=(FloatText(value=0.001, description='Timestep [s]'), FloatText(value=0.1, description='Du…